In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../GillesPy2')))

In [2]:
import json
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,mark_inset)

In [4]:
from gillespy2 import TauHybridCSolver

In [5]:
from dask.distributed import Client
from dask import delayed, compute

In [6]:
from Devils_DFTD_2_Stage_Infection import DevilsDFTD2StageInfection

In [7]:
c = Client(n_workers=2, threads_per_worker=1)

## Read in observed data

In [8]:
pop_data = pd.read_csv('Devils_Dataset__Population_1985-2020.csv')
devil_pop = np.array(pop_data['Population'].iloc[:].values)

obs = np.vstack([devil_pop]).reshape(1, 1, -1)
carry_cap = int(max(devil_pop)*1.16)

dates = []
year = 1985
while len(dates) < len(devil_pop):
    for month in ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"]:
        dates.append(f"{month} {year}")
    year += 1

In [9]:
sweeps={}
mean_std_db={}
type_variables = {
    "juvenile_concentration": float,
    "birth_rate": float,
    "maturity_rate": float,
    "death_rate_juvenile": float,
    "death_rate_susceptible": float,
    "death_rate_over_population": float,
    "infection_rate_infected": float,
    "infection_rate_diseased": float,
    "incubation": float,
    "progression": float,
    "death_rate_infected": float,
    "death_rate_diseased": float,
    "DFTD_start": float,
}

In [10]:
def setup_sweep():
    for name, v_type in type_variables.items():
        sweeps[name] = [variables[name]]
        
        sweeps[name].insert(0, variables[name] * 0.975)
        sweeps[name].insert(0, variables[name] * 0.95)
        sweeps[name].insert(0, variables[name] * 0.9)
        sweeps[name].insert(0, variables[name] * 0.85)
        sweeps[name].insert(0, variables[name] * 0.8)
        
        sweeps[name].append(variables[name] * 1.025)
        sweeps[name].append(variables[name] * 1.05)
        sweeps[name].append(variables[name] * 1.1)
        sweeps[name].append(variables[name] * 1.15)
        sweeps[name].append(variables[name] * 1.2)
            
        sweeps[name] = sorted(list(set(sweeps[name])))
        for n,x in enumerate(sweeps[name]):
            if type_variables[name]==int:
                x = str(x)
            else:
                x = f"{x:.4e}"
            if '00000' in x:
                x=re.sub(r"00000\d+",'',x)
            elif '99999' in x:
                x=re.sub(r"99999\d+",'9',x)
            sweeps[name][n]=x

In [11]:
def generate_all_space(variables, sweeps):
    # first add the base point
    all_space =[variables.copy()]
    # then go through each sweep and add the points
    for sn, svs in sweeps.items():
        for sv in svs:
            if variables[sn] == sv: continue # don't re-add base point
            v = variables.copy()
            if sn == "juvenile_concentration":
                init_Devils_pop = round(model.devil_pop[0])
                init_J_pop = round(model.devil_pop[0] * type_variables[sn](sv))
                init_S_pop = round(model.devil_pop[0] - init_J_pop)
                v["Juvenile"] = init_J_pop
                v["Susceptible"] = init_S_pop
            else:
                v[sn] = type_variables[sn](sv)
            life_span = 1/v['death_rate_diseased'] + v['incubation'] + v['progression']
            if life_span < 26 and life_span > 12:
                all_space.append(v)
    return all_space

In [12]:
def load_remote(var, sweeps, model, solver, batch_size=15):
    jobs = []
    keys = []
    all_space = generate_all_space(var, sweeps)
    for space in all_space:
        filename = make_filename(space)
        if os.path.exists(filename + ".m"):
            print(f"Found {filename}.m, skipping")
        
        job = delayed(model.run)(num_sims=1000, solver=solver, variables=space)
        batch = len(jobs)
        if batch == 0 or len(jobs[batch - 1]) == batch_size:
            jobs.append([job])
            keys.append([filename])
        else:
            jobs[batch-1].append(job)
            keys[batch-1].append(filename)
    return jobs, keys

In [13]:
def run_remote(model, jobs, keys, batch_size=15):
    total_sims = 0 if len(jobs) == 0 else batch_size * (len(jobs) - 1) + len(jobs[-1])
    print(f"Running {total_sims} new parameter points", end=" ")
    print(f"in {len(jobs)} batches with {batch_size} points per batch")
    
    for i, batch in enumerate(jobs):
        results = dict(zip(keys[i], compute(*batch)))
        
        for filename, result in results.items():
            (m, s) = model.calculate_distance(result)
            
            with open(f"{filename}.m", "w") as fd:
                fd.write(f"{m},{s}")

In [14]:
def make_filename(variables):
    cmdpath = "ParameterSweeps/tmp_result_state"
    ret = f"{cmdpath}/ps-"
    for k in sorted(variables.keys()):
        v = float(variables[k])
        x = f"{v:.4e}"
        ret += f"{x},"
    return ret

In [15]:
def get_mean_std(var):
    f = make_filename(var)
    if f in mean_std_db:
        return mean_std_db[f]
    
    file = f"{f}.m"
    try:
        with open(file, "r") as fd:
            resp = fd.read()
            (m,s) = resp.split(",",2)
            mean_std_db[f] = (float(m),float(s))
            return mean_std_db[f]
    except Exception as e:
        time.sleep(0.2)
        raise e

In [16]:
def plot_sweep(name):
    xvals = np.zeros(len(sweeps[name]))
    mvals = np.zeros(len(sweeps[name]))
    svals = np.zeros(len(sweeps[name]))
    for n,v in enumerate(sweeps[name]):
        c = variables.copy()
        c[name] = type_variables[name](v)
        keepgoing=True
        while keepgoing:
            try:
                (m,s) = get_mean_std(c)
                xvals[n]=c[name]
                mvals[n]=m
                svals[n]=s
                keepgoing=False
            except Exception as e:
                print(f"caught e={e} while name={name} ")
                time.sleep(1)
    
    plt.figure(figsize=[12, 6])
    plt.errorbar(xvals,mvals,yerr=svals, capsize=10)
    plt.plot([variables[name], variables[name]],[0, max(mvals)],'--')
    plt.xlabel(name, fontsize=12)
    plt.ylabel('Error', fontsize=12)

In [17]:
def plot_eresult(eresults): 
    fig, ax1 = plt.subplots(figsize=[12, 6])

    plt.title("Tasmanian Devil Population with DFTD: Observed vs. Simulated", fontsize=18)
    ax1.set_xlabel(f"Time (months) since {dates[0]}", fontsize=16)
    ax1.set_ylabel("Population of Tasmanian Devils", fontsize=16)
    ax1.plot(eresults[0]['time'], obs[0][0], '--', color='black', label='Observed Total')
    ax1.plot(eresults[0]['time'], eresults[0]['Devils'], color='blue', label='Simulated Total')
    ax1.plot(eresults[0]['time'], eresults[0]['Juvenile'], color='purple', alpha=.6, label='Juvenile')
    ax1.plot(eresults[0]['time'], eresults[0]['Susceptible'], color='green', alpha=.6, label='Susceptible')
    ax1.plot(eresults[0]['time'], eresults[0]['Exposed'], color='magenta', alpha=.6, label='Exposed')
    ax1.plot(eresults[0]['time'], eresults[0]['Infected'], color='red', alpha=.6, label='Infected')
    ax1.plot(eresults[0]['time'], eresults[0]['Diseased'], color='cyan', alpha=.6, label='Diseased')
    ax1.plot([variables['DFTD_start'], variables['DFTD_start']], [-3000, carry_cap], '--k', alpha=0.4)
    ax1.text(variables['DFTD_start']-7, 5000, "DFTD Start", rotation="vertical", color="black", fontsize=14)
    ax1.text(variables['DFTD_start']-7, 24000, dates[variables['DFTD_start']], rotation="vertical", color="black", fontsize=14)
    ax1.tick_params(axis='x', labelsize=12)
    ax1.set_yticks([20000,40000,60000])
    ax1.tick_params(axis='y',labelsize=12, labelrotation=90)
    ax1.legend(loc='upper right', fontsize=16)
    ax1.set_ylim([0, carry_cap])
    ax1.set_xlim(0,eresults[0]['time'][-1])
    for n,r in enumerate(eresults):
        if n==0: continue
        ax1.plot(eresults[n]['time'], eresults[n]['Devils'],'b', alpha=0.025)
        ax1.plot(eresults[n]['time'], eresults[n]['Juvenile'], color='purple', alpha=0.025, label='Juvenile')
        ax1.plot(eresults[n]['time'], eresults[n]['Susceptible'], color='green', alpha=0.025, label='Susceptible')
        ax1.plot(eresults[n]['time'], eresults[n]['Exposed'], color='magenta', alpha=0.025, label='Exposed')
        ax1.plot(eresults[n]['time'], eresults[n]['Infected'], color='red', alpha=0.025, label='Infected')
        ax1.plot(eresults[n]['time'], eresults[n]['Diseased'], color='cyan', alpha=0.025, label='Diseased')

    print(variables)
    
    def calculate_distance(eresults):
        '''return mean/stddev of L2 norm distance'''
        global obs
        dists = np.zeros(len(eresults))
        for n,r in enumerate(eresults):
            dists[n] = np.linalg.norm(r['Devils']-obs[0][0],2)
        return np.average(dists), np.std(dists)

    dist_l2 = calculate_distance(eresults)
    print(f"L2 = {dist_l2[0]:.2f} +/-{dist_l2[1]:.2f}")

# Run Parameter Sweep

In [18]:
variables = {
    'juvenile_concentration': 0.49534348836011316,
    'birth_rate': 0.055, 
    'maturity_rate': 0.04,#0.04167,
    'death_rate_juvenile': 0.007,#0.006, 
    'death_rate_over_population': 2.3e-07, 
    'death_rate_susceptible': 0.02335, 
    'incubation': 10.25,#10, #10.99687624550675, 
    'progression': 10.74,#11.015,#10.746230534983676,
    'infection_rate_diseased': 3.84e-05,#3.2e-05, #3.4e-05 #3e-05, #4.978182435648742e-05, 
    'infection_rate_infected': 1e-05,#1.0698e-05,#1.1261e-05,#1.155e-05,#1.1e-05, #1.2e-05 #1e-5, #1.4809664001475363e-05, 
    'death_rate_diseased': 0.29017,#0.25232,#0.22938,#0.23526,#0.27678,#0.29134996217062514, 
    'death_rate_infected': 0.022609,#0.01966,#0.020695079156445997, 
    'DFTD_start': 100,#97,#95, #105, 
}

variables_orig = variables.copy()

In [19]:
setup_sweep()

In [20]:
sweeps['DFTD_start']=['85','90','92','94','96','98','100','102','104','106','108','110','115']

In [21]:
print(json.dumps(variables, sort_keys=True, indent=4))

{
    "DFTD_start": 100,
    "birth_rate": 0.055,
    "death_rate_diseased": 0.29017,
    "death_rate_infected": 0.022609,
    "death_rate_juvenile": 0.007,
    "death_rate_over_population": 2.3e-07,
    "death_rate_susceptible": 0.02335,
    "incubation": 10.25,
    "infection_rate_diseased": 3.84e-05,
    "infection_rate_infected": 1e-05,
    "maturity_rate": 0.04,
    "progression": 10.74
}


In [22]:
model = DevilsDFTD2StageInfection()

In [23]:
solver = delayed(TauHybridCSolver)(model=model, variable=True)

In [24]:
%time jobs, keys = load_remote(var=variables, sweeps=sweeps, model=model, solver=solver)

Found ParameterSweeps/tmp_result_state/ps-1.0000e+02,5.5000e-02,2.9017e-01,2.2609e-02,7.0000e-03,2.3000e-07,2.3350e-02,1.0250e+01,3.8400e-05,1.0000e-05,4.0000e-02,1.0740e+01,.m, skipping
Found ParameterSweeps/tmp_result_state/ps-1.0000e+02,5.5000e-02,2.9017e-01,2.2609e-02,7.0000e-03,2.3000e-07,2.3350e-02,1.0250e+01,3.8400e-05,1.0000e-05,4.0000e-02,1.0740e+01,.m, skipping
Found ParameterSweeps/tmp_result_state/ps-1.0000e+02,5.5000e-02,2.9017e-01,2.2609e-02,7.0000e-03,2.3000e-07,2.3350e-02,1.0250e+01,3.8400e-05,1.0000e-05,4.0000e-02,1.0740e+01,.m, skipping
Found ParameterSweeps/tmp_result_state/ps-1.0000e+02,5.5000e-02,2.9017e-01,2.2609e-02,7.0000e-03,2.3000e-07,2.3350e-02,1.0250e+01,3.8400e-05,1.0000e-05,4.0000e-02,1.0740e+01,.m, skipping
Found ParameterSweeps/tmp_result_state/ps-1.0000e+02,5.5000e-02,2.9017e-01,2.2609e-02,7.0000e-03,2.3000e-07,2.3350e-02,1.0250e+01,3.8400e-05,1.0000e-05,4.0000e-02,1.0740e+01,.m, skipping
Found ParameterSweeps/tmp_result_state/ps-1.0000e+02,5.5000e-02,2

In [ ]:
%time run_remote(model=model, jobs=jobs, keys=keys)

Running 132 new parameter points in 9 batches with 15 points per batch


In [ ]:
%%time
for k in sweeps.keys():
    plot_sweep(k)

In [ ]:
model1 = DevilsDFTD2StageInfection(devil_fitting=True)
job1 = delayed(model1.run)(num_sims=100, solver=solver, variables=variables)
%time eresults1 = job1.compute()
plot_eresult(eresults1)

In [ ]:
final_devil_pop = eresults1[0]['Devils'][-1]

print(f"Number of Devils {final_devil_pop} at simulation end,",
      f"{final_devil_pop/carry_cap * 100:.1f}% of target carrying capacity")

In [ ]:
final_devil_pops = []
for n, r in enumerate(eresults1):
    final_devil_pops.append(r['Devils'][-1])
    
print(f"Number of Devils = {np.average(final_devil_pops):.0f} +/-"
      f"{np.std(final_devil_pops):.0f} at simulation end,",
      f"{np.average(final_devil_pops)/carry_cap*100:.1f}% of target carrying capacity")

In [ ]:
sim_data1 = {
    "variables": variables, "sweeps": sweeps, "eresults": eresults1, "mean_std_db": mean_std_db
}
print(sim_data1)
# with open('figure_data/Fig3A_Fig14A-data.p','wb+') as fd:
#     pickle.dump(sim_data1, fd)

In [ ]:
model2 = DevilsDFTD2StageInfection()
job2 = delayed(model2.run)(num_sims=100, solver=solver, variables=variables)
%time eresults2 = job2.compute()
plot_eresult(eresults2)

In [ ]:
sim_data2 = {
    "variables": variables, "sweeps": sweeps, "eresults": eresults2, "mean_std_db": mean_std_db
}
print(sim_data2)
# with open('figure_data/Fig3B_Fig14B-C-data.p','wb+') as fd:
#     pickle.dump(sim_data2, fd)